In [1]:
from cmath import nan
# from unicodedata import name
# from matplotlib import projections
# from matplotlib.pyplot import table
import pandas as pd
import os
import glob
from pymongo import MongoClient, ASCENDING, DESCENDING, errors
from datetime import date, timedelta, datetime, timezone
from flask import Flask, jsonify, request, redirect, Response, send_file
from flask_cors import CORS
import json
import math
# from urllib.parse import urlparse
# from urllib. parse import parse_qs
from flask_cors import CORS, cross_origin
from flask import send_from_directory
from pandas.tseries.offsets import MonthEnd
from werkzeug .utils import secure_filename
from zipfile import ZipFile
from svs_report import svsreport
from svs_report import gen_all_letters
from svs_report import gen_excel
import shutil
from bson import json_util, ObjectId

In [3]:
folder_path = 'output/letter doc/' + '1' + '/' + \
    '2' + '/' + '3' + '_to_' + '4'
# dir_list = os.listdir(folder_path)

archived = shutil.make_archive(
    "output/ZipFiles/"+'1'+'/'+'2'+'/'+'3'+'_to_'+'4'+"/Letters", 'zip', folder_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'output/letter doc/1/2/3_to_4'

In [1]:
import requests
import json
from statistics import mean
import datetime
from pymongo import MongoClient, ASCENDING
import schedule
import time

# MongoDB connection string
CONNECTION_STRING = "mongodb://mongodb0.erldc.in:27017,mongodb1.erldc.in:27017,mongodb10.erldc.in:27017/?replicaSet=CONSERV"

# API details
API_KEY = "6dc32d47-f46a-45c9-afaf-c6ce73c4ca71"
URL = f"https://gateway.grid-india.in/POSOCO/reports/1.0/WebAccessAPI/GetUtilityExternalSharedData?apikey={API_KEY}"
AUTH = ("erldc_internal_prod", "ErldcPr0d@052024")


def push_data():
    target_date = datetime.datetime.today().strftime("%d-%m-%Y")
    data = {
        "Date": target_date,
        "SchdRevNo": -1,
        "UserName": "erldc_internal_prod",
        "UtilAcronymList": [
            "BIHAR_STATE", "JHANKHAND_STATE", "DVC_STATE",
            "ODISHA_STATE", "WB_STATE", "SIKKIM_STATE"
        ],
        "UtilRegionIdList": [1]
    }

    # API call
    try:
        res = requests.post(URL, json=data, auth=AUTH, timeout=60)
        res.raise_for_status()
    except Exception as e:
        print(f"❌ API request failed: {e}")
        return

    try:
        a = res.json()["ResponseBody"]["GroupWiseDataList"]
    except Exception:
        print("❌ Invalid API response")
        return

    GNA_List = []
    for state_data in a:
        for GNA_Contract in state_data["GNAContractReqList"]:
            doc = {
                "FromDate": GNA_Contract["ContractFromDate"],
                "ToDate": GNA_Contract["ContractToDate"],
                "Gen_type": GNA_Contract["SellerGeneratorTypeName"],
                "Gen_Fuel": GNA_Contract["SellerGeneratorSubTypeName"],
                "Buyer_state": GNA_Contract["BuyerParentStateUtilAcronym"].strip(),
                "Seller": GNA_Contract["SellerAcronym"],
                "Contract_amount": mean(GNA_Contract["GNAContractReqJsonData"]["ContractAmount"]),
                "Link": GNA_Contract["LinkName"],
                "From_Region": GNA_Contract["LinkName"].split("-")[0],
                "InsertedOn": datetime.datetime.today().strftime("%d-%m-%Y") 
            }
            GNA_List.append(doc)

    # Push to MongoDB
    client = MongoClient(CONNECTION_STRING)
    db = client["mis"]
    energy_data = db["energy_data"]

    # Create unique index (only once; ignored if already exists)
    energy_data.create_index(
        [
            ("FromDate", ASCENDING),
            ("ToDate", ASCENDING),
            ("Buyer_state", ASCENDING),
            ("Seller", ASCENDING),
            ("Gen_type", ASCENDING),
            ("Gen_Fuel", ASCENDING),
            ("Contract_amount", ASCENDING),
        ],
        unique=True
    )

    # Insert with upsert (no duplicates)
    inserted, updated = 0, 0
    for doc in GNA_List:
        query = {
            "FromDate": doc["FromDate"],
            "ToDate": doc["ToDate"],
            "Buyer_state": doc["Buyer_state"],
            "Seller": doc["Seller"],
            "Gen_type": doc["Gen_type"],
            "Gen_Fuel": doc["Gen_Fuel"],
            "Contract_amount": doc["Contract_amount"]
        }
        result = energy_data.update_one(query, {"$set": doc}, upsert=True)
        if result.upserted_id:
            inserted += 1
        elif result.modified_count > 0:
            updated += 1

    print(f"✅ Job done at {datetime.datetime.now()} | Inserted: {inserted}, Updated: {updated}")

x=push_data()

# # Schedule jobs
# schedule.every().day.at("08:00").do(push_data)
# schedule.every().day.at("13:00").do(push_data)
# schedule.every().day.at("18:00").do(push_data)

# print("⏳ Scheduler started... Waiting for next run.")
# while True:
#     schedule.run_pending()
#     time.sleep(60)


✅ Job done at 2025-08-22 14:49:29.769474 | Inserted: 141, Updated: 0


In [ ]:
from flask import Flask, jsonify, request
from pymongo import MongoClient
from bson.json_util import dumps

app = Flask(__name__)

# MongoDB connection
CONNECTION_STRING = "mongodb://mongodb0.erldc.in:27017,mongodb1.erldc.in:27017,mongodb10.erldc.in:27017/?replicaSet=CONSERV"
client = MongoClient(CONNECTION_STRING)
db = client["mis"]
energy_data = db["energy_data"]


@app.route("/api/data", methods=["GET"])
def get_data():
    """
    Retrieve energy data from MongoDB.
    Optional query parameters:
      - Buyer_state
      - Seller
      - FromDate
      - ToDate
    """
    query = {}
    if "Buyer_state" in request.args:
        query["Buyer_state"] = request.args["Buyer_state"]
    if "Seller" in request.args:
        query["Seller"] = request.args["Seller"]
    if "FromDate" in request.args:
        query["FromDate"] = request.args["FromDate"]
    if "ToDate" in request.args:
        query["ToDate"] = request.args["ToDate"]

    results = energy_data.find(query, {"_id": 0})  # exclude MongoDB _id
    return dumps(results), 200, {"Content-Type": "application/json"}


@app.route("/api/data/<seller>", methods=["GET"])
def get_by_seller(seller):
    """Retrieve all contracts for a given Seller"""
    results = energy_data.find({"Seller": seller}, {"_id": 0})
    return dumps(results), 200, {"Content-Type": "application/json"}


@app.route("/api/states", methods=["GET"])
def list_states():
    """List all distinct Buyer states in DB"""
    states = energy_data.distinct("Buyer_state")
    return jsonify(states)


@app.route("/api/sellers", methods=["GET"])
def list_sellers():
    """List all distinct Sellers in DB"""
    sellers = energy_data.distinct("Seller")
    return jsonify(sellers)


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)